In this file we extract core-user and non core users modify the non core users and give to myncf such that its accuracy decreases

In [57]:
from fileinput import filename
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import svds
import random 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import scipy.stats as ss
import pickle
from sklearn.utils.extmath import randomized_svd

In [58]:
def ReadData():
    ml1m_dir = 'data/ratings.dat'
    ml1m_rating = pd.read_csv(ml1m_dir, sep='::', header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')
    unique_uid = np.unique(np.array(ml1m_rating['uid'].tolist()))
    unique_mid = np.unique(np.array(ml1m_rating['mid'].tolist()))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    mid_dict = dict([(y,x) for x,y in enumerate(unique_mid)])
    print('DICTIONARY PREPARED:')

    # init user item dictionary:
    
    uid_list = ml1m_rating['uid'].tolist()
    uid_list_len = len(uid_list)
    mid_list = ml1m_rating['mid'].tolist()
    mid_list_len = len(mid_list)
    rating_list = ml1m_rating['rating'].tolist()
    user_item_dict = {x:set() for x in range(len(unique_uid))}
    item_user_dict = {x:set() for x in range(len(unique_mid))}
    for i in range(uid_list_len):
        uid_list[i] = uid_dict[uid_list[i]]
        mid_list[i] = mid_dict[mid_list[i]]
        # rating_list[i] = 1 # comment this line if you want to activate explicit ratings
        user_item_dict[uid_list[i]].add(mid_list[i])
        item_user_dict[mid_list[i]].add(uid_list[i])
    tmp_df = pd.DataFrame({"uid":uid_list, "mid":mid_list, "ratings":rating_list})
    # v = tmp_df.uid.value_counts()
    # df = tmp_df[tmp_df.uid.isin(v.index[v.gt(30)])]
### code to store less than 30 interactions:
    # df_less_30 = tmp_df[tmp_df.uid.isin(v.index[v.le(30)])]
    return tmp_df, len(np.unique(mid_list)), len(unique_uid)

In [59]:
# threshold = 30 #split the users into test and train by threshold number of interactions. if greater than threshold then all interactions of that user goes into train set.
df, unique_mids, unique_uids = ReadData()
# print("GREATER THAN 30:\n", df_gt_30)
# print("LESS THAN 30: \n", df_le_30)
# print(len(df_gt_30))
# print(len(df_le_30))
print("UNIQUE MIDS: ", unique_mids)
print("UNIQUE UIDS: ", unique_uids)


DICTIONARY PREPARED:
UNIQUE MIDS:  3706
UNIQUE UIDS:  6040


In [4]:
# support_test_df = df_gt_30.groupby("uid").tail(1)
# # print(len(df_gt_30))
# support_train_df = df_gt_30.drop(df_gt_30.groupby('uid').tail(1).index, inplace=False)
# # print("#TEST INSTANCES: " ,len(support_test_df))
# # print("#TRAIN INSTANCES: " ,len(support_train_df))
# assert(len(df_gt_30)== len(support_test_df) + len(support_train_df))
# # print(len(test_df))
# # print(len(train_df))
# query_test_df = df_le_30.groupby("uid").tail(1)
# query_train_df = df_le_30.drop(df_le_30.groupby('uid').tail(1).index, inplace=False)
# assert(len(df_le_30)== len(query_test_df) + len(query_train_df))
# dic_support_train_df_uid_mapping = dict([(y,x) for x,y in enumerate(np.unique(support_train_df['uid']))])
# dic_support_train_df_uid_rmapping = dict([(x,y) for x,y in enumerate(np.unique(support_train_df['uid']))])
# ### no need for mid mapping

# uid_of_train_df = support_train_df['uid'].tolist()
# for i in range(len(uid_of_train_df)):
#     uid_of_train_df[i] = dic_support_train_df_uid_mapping[uid_of_train_df[i]]
# # for index, row in train_df.iterrows():
# #     train_df['uid'][index] = dic_train_df_uid_mapping[train_df['uid'][index]]
# core_user_ko_input_train_df = pd.DataFrame({'uid':uid_of_train_df, 'mid':support_train_df['mid'], 'ratings':support_train_df['ratings']})

In [5]:
# train_ui_dic = {}    
# for user in range(unique_uids):
#     train_ui_dic[user] = []
# for index,row in support_train_df.iterrows():
#         train_ui_dic[row['uid']].append(row['mid'])

- utility functions for CUR coreusers.

In [60]:
def ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)
    
    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])
    
    for index, row in dataframe.iterrows():
        userItemMatrix[row['uid']][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    df = pd.DataFrame(userItemMatrix)
    cosineSimilarity = cosine_similarity(df)
    print("SHAPE OF COSINE MATIX:\n ", cosineSimilarity.shape)

    listToStoreTopFiftyOfEveryUser = []
    for i in range(0, cosineSimilarity.shape[0]):
        idx = np.argpartition(cosineSimilarity[i], -50)[-50:]
        listToStoreTopFiftyOfEveryUser.append(idx)
    # print("Top fifty list: \n", listToStoreTopFiftyOfEveryUser)
    # listToStoreTopFiftyOfEveryUser = np.array(listToStoreTopFiftyOfEveryUser)
    flatten = np.concatenate(listToStoreTopFiftyOfEveryUser)
    listToStoreTopFiftyOfEveryUser = flatten.ravel()

    # print("List of top 50", listToStoreTopFiftyOfEveryUser)
    df = pd.DataFrame(listToStoreTopFiftyOfEveryUser)
    allUserList = df.value_counts().index.tolist()
    # print("ALL USERS LIST", allUserList)
    allUserList = list(sum(allUserList,()))
    # print("ALL USERS LIST", allUserList)
    twentyPercentUserList = allUserList[:int(len(allUserList)*0.2)]
    # print("TWENTY PERCENT USER:", len(twentyPercentUserList))
    # print("TWENTY PERCENT USER:", (twentyPercentUserList))
    
    r_ind = twentyPercentUserList
    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cos_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # coreusers = dataframe.iloc[np.where(dataframe.uid.isin(twentyPercentUserList))]
    # coreusers.reset_index()
    # print("CORE USERS:\n", coreusers)
    return cos_coreusers

In [61]:
MAX_MID = 27277 + 1
def select_cols(mat, k, dup=False):
    # prob 1d array of probabilities of all columns
    prob = mat.T.dot(mat)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis = 0)
    prob = prob/denom

    C = np.zeros((mat.shape[0], k))
    ind_cols = np.arange(0, prob.size)
    c_ind = []
    i = 0
    while(i < k):
        rand_sel = np.random.choice(ind_cols, 1, p=prob)
        if rand_sel in c_ind:
            continue
        c_ind.append(rand_sel[0])
        C[:, i] = mat[:, rand_sel[0]]
        i += 1
        # C[:, i] = C[:, i]/np.sqrt(k*prob[rand_sel[0]])

    return C, c_ind

def select_rows(mat, k, dup=False):

    prob = mat.dot(mat.T)
    prob = np.array(np.diagonal(prob))
    denom = np.abs(prob).sum(axis=0)
    prob = prob/denom
    print(prob)
    r = np.zeros((k, mat.shape[1]))
    ind_rows = np.arange(0, prob.size)
    r_ind = []
    i = 0
    while(i < k):
        # print(ind_rows)
        rand_sel = np.random.choice(ind_rows, 1, p=prob)
        if rand_sel in r_ind:
            continue
        r_ind.append(rand_sel[0])
        r[i, :] = mat[rand_sel[0], :]
        i += 1
        # r[i, :] = r[i, :]/np.sqrt(k*prob[rand_sel[0]])
    r_ind = np.array(r_ind)
    return r, r_ind

# def matIntersection(mat, c_ind, r_ind):
    
#     W = np.zeros((len(r_ind), len(c_ind)))
#     for i in range(len(r_ind)):
#         W[i] = mat[r_ind[i], c_ind]
    
#     return W

# def pseudoInverse(W):
#     # U = WP (W+)

#     # W = X.Z.YT
#     X, Z, YT = np.linalg.svd(W)
    
#     # W+ = Y.Z+.XT
#     XT = X.T
#     Y = YT.T
#     # Z+ = reciprocal(Z)
#     ZP = np.reciprocal(Z)
#     ZP = sp.spdiags(ZP, 0, ZP.size, ZP.size)
#     ZP = ZP@ZP
    
#     # W+ = Y.Z+.XT
#     WP = Y@ZP
#     WP = WP@XT

#     return WP

In [62]:
def CUR_ExtractCoreUsers(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[row['uid']]][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    mat = userItemMatrix
    print("MAT:", mat)
    print(mat.shape)
    C, c_ind = select_cols(mat, int(u_len * percent_core_user)) ## getting 20% core users
    r, r_ind= select_rows(mat, int(u_len * percent_core_user))
    # print("r", r)
    # print("r_ind len", len(r_ind))

    len_r_ind = len(r_ind)
    new_r_ind = [rev_uid_dict[r_ind[i]] for i in range(len_r_ind)]
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(new_r_ind))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

In [63]:
def RandomSampling(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)

    how_much_to_sample = int(percent_core_user * u_len)
    indices = random.sample(range(u_len), how_much_to_sample)
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(indices))]
    return cur_coreusers

In [64]:
class largest_leveragescores_Sampler:
    def __init__(self, A, k, Q,N):
        """ Create largest k-leverage scores Sampler for the matrix :math:`A` for k-low rank apparoximation.
        :param A: 
            Matrix :math:`A`.
        :type A: 
            array_type
        :param Q: 
            Matrix containig the right singular vectors of :math:`A`.
        :type Q: 
            array_type
        :param k: 
            The order of low rank apparoximation.
        :type k: 
            int
        :param N: 
            The dimension of subsampling (the number of columns) of A.
        :type N: 
            int
        """
        self.A = A
        # print("k is", type(k))
        self.Q = np.transpose(Q[0:k,:])
        self.N = N
        self.k = k
        
        self.sampling_list = []
        self.lvs_array = self.Estimate_Leverage_Scores()
    def Estimate_Leverage_Scores(self):
        return 1/(self.k)*np.diag(np.dot(self.Q,self.Q.T))
    def MultiRounds(self):
        temp_list = list(reversed(np.argsort(self.lvs_array)))
        sampled_indices_ = temp_list[0:self.k]
        self.sampling_list = sampled_indices_
        return sampled_indices_
from scipy.sparse.linalg import svds
def Extract_LLS(dataframe, unique_user_len, unique_item_len, percent_core_user=0.40):
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    
    # print(userItemMatrix)

    unique_uid = np.unique(np.array(dataframe['uid'].tolist()))
    assert(u_len == len(unique_uid))
    # print("UNIQUE UID:", unique_uid)
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    rev_uid_dict = dict([(x,y) for x,y in enumerate(unique_uid)])

    userItemMatrix = np.zeros(shape=(u_len, m_len))
    for index, row in dataframe.iterrows():
        userItemMatrix[uid_dict[int(row['uid'])]][int(row['mid'])] = int(row['ratings'])
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)
    A = userItemMatrix.T
    number_of_coreusers = int(u_len * percent_core_user)
    print(number_of_coreusers)
    u4, s4, Q = svds(A, k=number_of_coreusers)
    lls = largest_leveragescores_Sampler(A, number_of_coreusers, Q, number_of_coreusers)
    indx = lls.MultiRounds()
    # indx = indx.
    print("LENGTH OF INDEX:", len(indx))
    print("INDICES ARE:", indx)
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(indx))]
    # # coreusers.reset_index()
    # # print("CORE USERS:\n", coreusers)
    return cur_coreusers

- cur decomposition from paper (https://www.pnas.org/doi/10.1073/pnas.0803205106) called optimal cur

In [65]:
class CUR():
    def __init__(self, k, eps, it=None, truncated=False):
        self.k = k
        print("K IS :", self.k)
        self.eps = eps
        self.trunc = truncated 
        self.c = k * np.log(k) / eps**2 #expectation number of sampled columns
        self.C, self.U, self.R = None, None, None #matrices of decomposition
        self.pi_col, self.pi_row = None, None #leverage scores of corresponding columns/rows
        self.col_indices = None
        self.row_indices = None
    
    def column_select(self, A):
        n = A.shape[1]
        A = np.array(A.copy())
        if self.trunc:
            _, _, v_k = randomized_svd(A, self.k) #for very big matrices
        else:
            _, _, vh = np.linalg.svd(A, full_matrices=False)
            v_k = vh[0:self.k, :]
        
        pi = 1 / self.k * np.sum(v_k**2, axis=0)
        c_index = [np.random.choice(2, 
                        p=[1 - min(1, self.c * pi[i]), min(1, self.c * pi[i])]) for i in range(n)
                  ]
        c_index = np.nonzero(c_index)[0]
        print(len(c_index))
        C = A[:, c_index]
        return C, c_index, pi

    def run_CUR(self, A):
        A = np.array(A.copy())
        # self.C, self.col_indices, self.pi_col = self.column_select(A)
        self.R, self.row_indices, self.pi_row = self.column_select(A.T)
        # self.U = np.linalg.pinv(self.C) @ A @ np.linalg.pinv(self.R.T)
        return self.row_indices
def OPTIMAL_CUR_ExtractCoreUsers(dataframe, unique_user_len, unique_item_len):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)

    for index, row in dataframe.iterrows():
        userItemMatrix[row['uid']][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    mat = userItemMatrix
    print("MAT:", mat)
    print(mat.shape)
    cur = CUR(k=int(u_len * 0.10), eps=0.5,)
    # ids = np.argsort(cur.pi_col)[::-1][:5]
    cur.run_CUR(mat)
    cur_coreusers_idx = np.argsort(cur.pi_row)[::-1][:int(u_len * 0.10)]
    cur_coreusers = dataframe.iloc[np.where(dataframe.uid.isin(cur_coreusers_idx))]
    # print(len(cur_coreusers))
    return cur_coreusers

In [67]:
# core_users = CUR_ExtractCoreUsers(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids, percent_core_user=0.40)
# sdf.to_csv('file_name.csv', index=False)
# core_users = ExtractCoreUsers(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids)
# core_users = RandomSampling(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids, percent_core_user=0.40)
core_users = Extract_LLS(df, len(np.unique(df['uid'])), unique_mids, percent_core_user=0.40)
support_user_list = np.unique(core_users['uid'])
print("CORE USERS:" ,core_users)
print("NUMBER OF CORE USERS:", len(support_user_list))

USER LEN: 6040
MOVIE LEN: 3706
USER ITEM MATRIX: 
 [[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
2416


### CODE SNIPPET FOR GETTING NESTED CORE USERS:

In [13]:
core_users_nested = CUR_ExtractCoreUsers(core_users, len(np.unique(core_users['uid'])), unique_mids, percent_core_user=0.50)
print(len(np.unique(core_users_nested['uid'])))
print("NESTED CORE USERS:\n" ,core_users_nested)

USER LEN: 2416
MOVIE LEN: 3706
USER ITEM MATRIX: 
 [[0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [5. 5. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
MAT: [[0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [5. 5. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
(2416, 3706)
[0.00020753 0.00020781 0.00066301 ... 0.00099564 0.00027636 0.0004537 ]
1208
NESTED CORE USERS:
          uid   mid  ratings
254        4  2775        4
255        4  2140        4
256        4  1087        5
257        4    38        3
258        4   279        2
...      ...   ...      ...
999720  6036  1008        4
999721  6036  1014        4
999722  6036  1017        3
999723  6036   548        4
999724  6036  1025        5

[474791 rows x 3 columns]


#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [43]:
core_users_index_list_nested = core_users_nested.index.to_list()
non_core_user_index = (df.index.difference(core_users_nested.index))
non_core_user_index = non_core_user_index.tolist()

core_users_df_nested = df.loc[core_users_index_list_nested]
non_core_user_df = df.loc[non_core_user_index]
print("NON CORE USERS:" ,non_core_user_df)
print("CORE USERS:", core_users_nested)
print(len(np.unique(non_core_user_df['uid'])))

NON CORE USERS:           uid   mid  ratings
0           0  1104        5
1           0   639        3
2           0   853        3
3           0  3177        4
4           0  2162        5
...       ...   ...      ...
1000204  6039  1019        1
1000205  6039  1022        5
1000206  6039   548        5
1000207  6039  1024        4
1000208  6039  1025        4

[525418 rows x 3 columns]
CORE USERS:          uid   mid  ratings
254        4  2775        4
255        4  2140        4
256        4  1087        5
257        4    38        3
258        4   279        2
...      ...   ...      ...
999720  6036  1008        4
999721  6036  1014        4
999722  6036  1017        3
999723  6036   548        4
999724  6036  1025        5

[474791 rows x 3 columns]
4832


In [54]:
uid_of_core_users = np.unique(core_users_df_nested['uid']).tolist()
uid_non_core_users = np.unique(non_core_user_df['uid']).tolist()
import pickle
with open("ml-1m_core_users_and_non_core_users_obj.pkl", "wb") as f:
    pickle.dump(uid_of_core_users, f)
    pickle.dump(uid_non_core_users, f)

In [55]:
## generate original matrix for non core_user df:
matrix = np.zeros((unique_uids, unique_mids))
for index, row in non_core_user_df.iterrows():
    matrix[int(row['uid'])][int(row['mid'])] = int(1)
## write this numpy matrix to a file
with open("ml-1m_non_core_user_numpy_matrix.pkl", "wb") as f:
    pickle.dump(matrix, f)
    pickle.dump(unique_uids, f)
    pickle.dump(unique_mids, f)



In [45]:
support_test_df = core_users_df_nested.groupby("uid").tail(1)
# print(len(df_gt_30))
support_train_df = core_users_df_nested.drop(core_users_df_nested.groupby('uid').tail(1).index, inplace=False)
# print("#TEST INSTANCES: " ,len(support_test_df))
# print("#TRAIN INSTANCES: " ,len(support_train_df))
assert(len(core_users_df_nested) == len(support_test_df) + len(support_train_df))
# print(len(test_df))
# print(len(train_df))
query_test_df = non_core_user_df.groupby("uid").tail(1)
query_train_df = non_core_user_df.drop(non_core_user_df.groupby('uid').tail(1).index, inplace=False)
assert(len(non_core_user_df)== len(query_test_df) + len(query_train_df))

In [46]:
train_ui_dic = {}    
for user in range(unique_uids):
    train_ui_dic[user] = []
for index,row in support_train_df.iterrows():
        train_ui_dic[row['uid']].append(row['mid'])

#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [47]:
unique_uids_in_support_trian = np.unique(np.array(core_users_df_nested['uid']))
unique_uids_in_query_trian = np.unique(query_train_df['uid'])
print(len(unique_uids_in_support_trian))
support_test_df = support_test_df.loc[support_test_df['uid'].isin(unique_uids_in_support_trian)]
print("SUPPORT TEST DF:" ,support_test_df)
query_test_df = query_test_df
print("QUERY TEST DF:\n", query_test_df)

1208
SUPPORT TEST DF:          uid   mid  ratings
451        4   683        4
1199       9  1868        5
1336      10  1152        4
1939      16  2773        5
2244      17  1154        5
...      ...   ...      ...
995572  6010  1865        5
995719  6012   548        5
996791  6015  3540        4
997442  6022  1017        1
999724  6036  1025        5

[1208 rows x 3 columns]
QUERY TEST DF:
           uid   mid  ratings
52          0  1154        4
181         1  1155        5
232         2  1900        4
253         3  1148        5
522         5    33        4
...       ...   ...      ...
998634   6034  1865        4
999522   6035  1867        3
999744   6037  1007        5
999867   6038  1025        4
1000208  6039  1025        4

[4832 rows x 3 columns]


#### CODE SNIPPET FOR GETTING NESTED CORE USER

In [48]:
support_train = []
for index,row in core_users_df_nested.iterrows():
    support_train.append([row['uid'], row['mid'], row['ratings']])
query_train = []
for index, row in query_train_df.iterrows():
    query_train.append([row['uid'], row['mid'], row['ratings']])
support_test = []
for index, row in support_test_df.iterrows():
    support_test.append([row['uid'], row['mid'], row['ratings']])
query_test = []
for index, row in query_test_df.iterrows():
    query_test.append([row['uid'], row['mid'], row['ratings']])
user_his_dic = {}
for u in train_ui_dic.keys():
    user_his_dic[u] = train_ui_dic[u]
user_supp_list = np.unique(core_users_df_nested['uid']).tolist()

In [49]:
import pickle
with open("input_for_getting_query_embeddings.pkl", "wb") as f:
    pickle.dump(support_train, f)
    pickle.dump(query_train, f)
    pickle.dump(support_test, f)
    pickle.dump(query_test, f)
    pickle.dump(user_supp_list, f)
    pickle.dump(user_his_dic, f)

Getting embeddings for query users and which we will use to mul

In [50]:
!python pretrain-1m.py
!python train-1m.py
# !python test-1m.py

-------Dataset Info--------
split way [threshold] with threshold 30 training_ratio 1.0
train set size: support/query 474791/520586
test set size: support/query 1208/4832
Epoch 0 Step 441: Train 1.9116 Reg: 0.5788
Test: 0.8034 MAE: 0.7135 RMSE: 0.8963
Val: 0.8192 MAE: 0.7161 RMSE: 0.9051
Epoch 1 Step 882: Train 0.8167 Reg: 0.4449
Test: 0.8020 MAE: 0.7115 RMSE: 0.8956
Val: 0.8153 MAE: 0.7145 RMSE: 0.9029
Epoch 2 Step 1323: Train 0.8128 Reg: 0.3585
Test: 0.7934 MAE: 0.7122 RMSE: 0.8907
Val: 0.8087 MAE: 0.7108 RMSE: 0.8993
Epoch 3 Step 1764: Train 0.8100 Reg: 0.3151
Test: 0.7999 MAE: 0.7133 RMSE: 0.8943
Val: 0.8075 MAE: 0.7120 RMSE: 0.8986
Epoch 4 Step 2205: Train 0.8054 Reg: 0.2917
Test: 0.8000 MAE: 0.7121 RMSE: 0.8944
Val: 0.8054 MAE: 0.7104 RMSE: 0.8974
Epoch 5 Step 2646: Train 0.7999 Reg: 0.2786
Test: 0.7875 MAE: 0.7031 RMSE: 0.8874
Val: 0.7976 MAE: 0.7039 RMSE: 0.8931
Epoch 6 Step 3087: Train 0.7880 Reg: 0.2771
Test: 0.7905 MAE: 0.7026 RMSE: 0.8891
Val: 0.7856 MAE: 0.6989 RMSE: 0.8863

In [56]:
!python test-1m.py

Extra : False
-------Dataset Info--------
split way [threshold] with threshold 30 training_ratio 1.0
train set size: support/query 474791/520586
test set size: support/query 1208/4832
CORE IS SELECTED:
SIZE OF MODEL: torch.Size([6040, 32])
SIZE OF MODEL: torch.Size([3706, 32])
torch.Size([6040, 3706])
MATRIX IS: tensor([[-4.7781e-06, -1.4933e-07,  4.3704e-07,  ...,  1.0380e-06,
         -5.8563e-08,  3.3805e-06],
        [ 7.1967e-07, -6.1768e-07,  6.8633e-07,  ..., -1.0391e-07,
          2.4735e-07,  1.3059e-06],
        [ 1.6817e-06, -4.0636e-07, -8.1827e-07,  ..., -9.3848e-07,
          7.9961e-07,  1.6096e-06],
        ...,
        [ 8.2051e-07, -6.3332e-07,  3.9873e-06,  ..., -5.2738e-07,
         -1.2242e-07,  1.2278e-06],
        [ 3.7712e-07, -3.9467e-07, -7.3393e-07,  ...,  2.1585e-07,
          2.7295e-07, -2.1748e-07],
        [-9.6813e-06, -1.4478e-06, -8.5960e-07,  ..., -1.2186e-07,
         -1.4770e-06,  1.8190e-06]], device='cuda:0')
NUMPY MATRIX: [[-4.7781068e-06 -1.493